<a href="https://colab.research.google.com/github/MTia05/Visualizzazione-Scientifica/blob/main/codici/confrontoFumatoriEta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import numpy as np
import os

# Percorso
# Trova la cartella dove si trova il notebook
base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Unisce i pezzi in modo corretto
file_path = os.path.join(base_path, "dataSet", "datasetFumatoriEta.csv")

try:
    # Caricamento tabella
    try:
        df = pd.read_csv(
            file_path,
            sep=',',
            encoding='utf-8',
            quoting=csv.QUOTE_NONE,
            on_bad_lines='skip'
        )
    except UnicodeDecodeError:
        df = pd.read_csv(
            file_path,
            sep=',',
            encoding='latin-1',
            quoting=csv.QUOTE_NONE,
            on_bad_lines='skip'
        )

    # Pulizia delle colonne
    df.columns = [c.strip().replace('"', '').replace("'", "") for c in df.columns]

    # Indici colonne basati sulla tabella
    # 9 - Età, 10 - TIME_PERIOD (Anno), 11 - Osservazione (Valore in migliaia)
    col_eta_idx = 9
    col_anno_idx = 10
    col_valore_idx = 11

    # Caratteri accentati nelle etichette dell'età
    data = pd.DataFrame({
        'Età': df.iloc[:, col_eta_idx].str.replace('"', '').str.strip(),
        'Anno': df.iloc[:, col_anno_idx],
        'Fumatori': df.iloc[:, col_valore_idx]
    })

    # Correzione della parola "più"
    data['Età'] = data['Età'].str.replace('pià', 'più').str.replace('piÃ¹', 'più')

    # Conversione tipi
    data['Anno'] = pd.to_numeric(data['Anno'], errors='coerce')
    data['Fumatori'] = pd.to_numeric(data['Fumatori'], errors='coerce')
    data = data.dropna()

    # Solo 2023 e 2024
    years = [2023, 2024]
    data = data[data['Anno'].isin(years)]

    # Estetica
    sns.set_theme(style="white")
    fig, ax = plt.subplots(figsize=(15, 8), dpi=100)

    # Colori
    color_2023 = "#1a237e" # Blu Profondo
    color_2024 = "#5DADE2" # Blu Cielo
    text_color = "#2E4053" # Titolo
    sub_color = "#95A5A6"  # Assi/Sottotitoli

    # Creazione barplot
    sns.barplot(
        data=data,
        x='Età',
        y='Fumatori',
        hue='Anno',
        palette={2023: color_2023, 2024: color_2024},
        ax=ax,
        edgecolor='white',
        linewidth=1
    )

    # Funzione per il "punto alto" e suffisso "k"
    def format_high_dot_k(value):
        # Virgola come separatore migliaia
        formatted = f'{int(value):,}'
        # Sostituisco la virgola con il punto alto e aggiungo k
        return formatted.replace(',', '˙') + 'k'

    # Valori sopra ogni colonna
    for container in ax.containers:
        # Genera i numeri
        labels = [format_high_dot_k(v.get_height()) for v in container]
        ax.bar_label(
            container,
            labels=labels,
            padding=5,
            fontsize=9,
            fontweight='bold',
            color=text_color
        )

    # Titolo
    plt.title('Distribuzione Fumatori per Fascia d\'Età',
              fontsize=22, pad=55, fontweight='bold', color=text_color, loc='left')

    # Sottotitolo
    ax.text(0, 1.07, 'Confronto 2023 vs 2024 • Valori espressi in migliaia',
            transform=ax.transAxes,
            fontsize=12,
            color=sub_color,
            fontweight='normal')

    # Nomi degli assi
    plt.xlabel('Fascia d\'Età', fontsize=11, labelpad=15, color=sub_color)
    plt.ylabel('Fumatori (migliaia)', fontsize=11, labelpad=15, color=sub_color)

    # Pulizia bordi
    sns.despine(left=True, bottom=True)

    # Griglia e formattazione asse Y
    ax.yaxis.grid(True, linestyle='-', alpha=0.1, color='black')

    # Formato anche all'asse Y
    def y_axis_formatter(x, pos):
        return format_high_dot_k(x)
    ax.yaxis.set_major_formatter(plt.FuncFormatter(y_axis_formatter))

    plt.xticks(fontsize=10, color=sub_color)
    plt.yticks(fontsize=10, color=sub_color)

    # Legenda
    leg = ax.legend(title='Anno', frameon=True, loc='upper right', bbox_to_anchor=(1, 1.12))
    leg.get_frame().set_linewidth(0.5)
    leg.get_frame().set_edgecolor('#D5DBDB')

    plt.tight_layout()
    plt.show()

except Exception as e:
    print(f"Errore durante l'elaborazione: {e}")